In [14]:
"""
词向量降维聚类
"""
import umap
import hdbscan
import pandas as pd
from sklearn import metrics
from sklearn.cluster import KMeans, AgglomerativeClustering

In [15]:
data = pd.read_excel('../data/mywv_wwm_tencent_sentence_count100.xlsx')
data

,noun,CBOW200,CBOW300,SG200,SG300,roberta_wwm,tencent200,sentence_transformer
0,交通,0.42336562 -1.1400625 -2.0374057 -1.9194353 0....,-0.06610227 0.4988237 0.13218886 -0.77638644 1...,-0.012540849 -0.38462433 -0.32319498 -0.182240...,0.023853732 -0.09414571 -0.23024663 0.03881392...,-0.08571069687604904 0.05334413796663284 0.758...,-0.097999 -0.172532 0.674114 -0.103633 0.24910...,-0.0059281155 0.085107535 -0.018252835 0.06734...
1,房间,0.28802317 -0.4507514 0.35016647 -0.43055606 0...,0.12421269 0.024913808 -0.01602313 1.1140127 -...,0.19959436 -0.32069674 -0.14715314 0.061708618...,0.17186686 -0.013962293 -0.13654591 0.1969577 ...,-0.26072731614112854 0.6660227179527283 0.2643...,0.181897 0.389742 -0.06113 0.242304 -0.028627 ...,0.01048636 -0.015002572 -0.017642647 -0.040445...
2,面积,1.1007886 -0.69985676 0.23117578 -1.8015103 -1...,0.44881797 -0.6757507 0.74121505 0.08760744 -0...,0.2180409 -0.31570902 0.16386202 -0.18676051 -...,-0.0023564755 0.14263394 -0.27440792 0.0326909...,-0.4051162004470825 0.6650103330612183 0.01912...,0.311055 0.039625 0.333128 0.215967 -0.111913 ...,-0.06143476 -0.044589818 -0.019351065 0.036645...
3,布局,0.7827691 -0.91875976 -1.3275683 0.19824812 -0...,-0.8795247 -0.32223016 0.24907054 -0.6157134 -...,0.24405675 -0.41571265 -0.17313722 0.071001776...,0.15417029 -0.06498914 -0.15444419 -0.1932353 ...,0.0831306055188179 0.5076727867126465 0.217217...,0.266127 0.316716 0.021962 0.075516 0.086151 -...,-0.037475087 -0.01469476 -0.02024162 0.0472815...
4,卫浴,0.60243356 -0.9301406 -0.243199 -0.7276225 -0....,-0.12032523 0.20740871 0.35552785 0.07866025 -...,0.25761604 -0.07487154 -0.19477549 0.028649133...,0.19327797 0.098093584 0.0371605 -0.029816944 ...,-0.3666742742061615 0.4476194381713867 0.67427...,0.230336 -0.213324 0.503645 0.260184 0.126454 ...,0.028435966 -0.098250374 -0.019702565 -0.08827...
...,...,...,...,...,...,...,...,...
935,三晚,-0.09600455 -0.24936976 0.08400188 0.089544974...,0.07487932 0.25357836 0.07852432 -0.0952891 0....,0.2406698 -0.5435254 0.10679261 0.0504834 -0.1...,0.123874895 -0.011844655 0.017515218 -0.050921...,0.22766144573688507 0.20426048338413239 0.3718...,-0.00788 0.306547 0.304058 0.261574 -0.089843 ...,0.005660566 -0.17151478 -0.012845519 0.0224691...
936,礼貌,1.1424041 0.91151 0.43261242 0.6441254 0.60611...,-0.46472156 0.32605383 -0.73007 0.9956819 -0.6...,0.30669054 -0.12392349 0.01986696 0.033570386 ...,-0.029739557 -0.0032241284 -0.1473059 0.151833...,-0.09216073155403137 0.6346340179443359 0.4351...,-0.324412 0.066763 0.139822 0.105918 -0.272539...,0.035092603 0.1269345 -0.019277968 0.072855875...
937,合适,-0.42161575 -0.19991678 1.7154243 -0.1863414 -...,0.836252 0.31240836 0.034565493 -0.46289173 0....,0.12132719 -0.42330444 0.15789798 0.16709346 0...,-0.21083379 0.13041505 -0.13174589 -0.09491256...,0.039119869470596313 0.3964913487434387 -0.013...,-0.266707 0.312565 -0.102081 0.449369 -0.25977...,0.021969374 0.031377662 -0.020527236 0.0181356...
938,威斯汀,-0.28125665 0.091284014 0.24053097 -0.07472294...,0.21199425 -0.13832602 -0.09146086 -0.09332652...,-0.3902142 -0.058192953 -0.1316207 -0.21724865...,0.15095249 0.24139097 -0.13767847 -0.099240825...,-0.4285953640937805 0.3718821406364441 0.07398...,-0.099088 0.133279 -0.222177 -0.11384 -0.31356...,-0.07128871 0.10444158 -0.018124782 0.08321853...


In [16]:
reducer = umap.UMAP(n_neighbors=15, n_components=5, metric='cosine', random_state=42)
metric = pd.DataFrame()
for name, column in data.iteritems():
    if name == 'noun':
        metric['metric'] = ['dvi', 'sc', 'chs', 'class_number']
        continue

    # 类型转换
    embedding_list = column.apply(lambda x: [float(number) for number in x.split(' ')]).tolist()

    # 降维
    reduced = reducer.fit_transform(embedding_list)

    # 聚类 k:kmeans, h:AgglomerativeClustering, hd:hdbscan
    reduced_k = KMeans(n_clusters=35, random_state=42).fit(reduced)
    original_k = KMeans(n_clusters=35, random_state=42).fit(embedding_list)
    reduced_h = AgglomerativeClustering(n_clusters=35).fit(reduced)
    original_h = AgglomerativeClustering(n_clusters=35).fit(embedding_list)
    reduced_hd = hdbscan.HDBSCAN(min_cluster_size=10, min_samples=1).fit(reduced)
    original_hd = hdbscan.HDBSCAN(min_cluster_size=4, min_samples=1).fit(embedding_list)

    # metric
    # hdbscan去掉噪音点计算metric
    non_noise_re, non_noise_labels_re = [], []
    for sample, label in zip(reduced, reduced_hd.labels_):
        if label != -1:
            non_noise_re.append(sample)
            non_noise_labels_re.append(label)

    non_noise, non_noise_labels = [], []
    for sample, label in zip(embedding_list, original_hd.labels_):
        if label != -1:
            non_noise.append(sample)
            non_noise_labels.append(label)

    dvi_re_k = metrics.davies_bouldin_score(reduced, reduced_k.labels_)
    sc_re_k = metrics.silhouette_score(reduced, reduced_k.labels_)
    chs_re_k = metrics.calinski_harabasz_score(reduced, reduced_k.labels_)

    dvi_k = metrics.davies_bouldin_score(embedding_list, original_k.labels_)
    sc_k = metrics.silhouette_score(embedding_list, original_k.labels_)
    chs_k = metrics.calinski_harabasz_score(embedding_list, original_k.labels_)

    dvi_re_h = metrics.davies_bouldin_score(reduced, reduced_h.labels_)
    sc_re_h = metrics.silhouette_score(reduced, reduced_h.labels_)
    chs_re_h = metrics.calinski_harabasz_score(reduced, reduced_h.labels_)

    dvi_h = metrics.davies_bouldin_score(embedding_list, original_h.labels_)
    sc_h = metrics.silhouette_score(embedding_list, original_h.labels_)
    chs_h = metrics.calinski_harabasz_score(embedding_list, original_h.labels_)

    dvi_re_hd = metrics.davies_bouldin_score(non_noise_re, non_noise_labels_re)
    sc_re_hd = metrics.silhouette_score(non_noise_re, non_noise_labels_re)
    chs_re_hd = metrics.calinski_harabasz_score(non_noise_re, non_noise_labels_re)

    dvi_hd = metrics.davies_bouldin_score(non_noise, non_noise_labels)
    sc_hd = metrics.silhouette_score(non_noise, non_noise_labels)
    chs_hd = metrics.calinski_harabasz_score(non_noise, non_noise_labels)

    # 保存聚类结果
    data[name + '_re_k'] = reduced_k.labels_
    data[name + '_k'] = original_k.labels_
    data[name + '_re_h'] = reduced_h.labels_
    data[name + '_h'] = original_h.labels_
    data[name + '_re_hd'] = reduced_hd.labels_
    data[name + '_hd'] = original_hd.labels_

    metric[name + '_re_k'] = [dvi_re_k, sc_re_k, chs_re_k, reduced_k.labels_.max() +1]
    metric[name + '_k'] = [dvi_k, sc_k, chs_k, original_k.labels_.max() + 1]
    metric[name + '_re_h'] = [dvi_re_h, sc_re_h, chs_re_h, reduced_h.labels_.max() + 1]
    metric[name + '_h'] = [dvi_h, sc_h, chs_h, original_h.labels_.max() + 1]
    metric[name + '_re_hd'] = [dvi_re_hd, sc_re_hd, chs_re_hd, reduced_hd.labels_.max() + 1]
    metric[name + '_hd'] = [dvi_hd, sc_hd, chs_hd, original_hd.labels_.max() + 1]

C:\Users\62774\anaconda3\envs\kansei\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\62774\anaconda3\envs\kansei\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\62774\anaconda3\envs\kansei\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\62774\anaconda3\envs\kansei\lib\site-packages\sklearn\cluster\_kmeans.py:1334: UserWarning:

In [17]:
data

,noun,CBOW200,CBOW300,SG200,SG300,roberta_wwm,tencent200,sentence_transformer,CBOW200_re_k,CBOW200_k,...,tencent200_re_h,tencent200_h,tencent200_re_hd,tencent200_hd,sentence_transformer_re_k,sentence_transformer_k,sentence_transformer_re_h,sentence_transformer_h,sentence_transformer_re_hd,sentence_transformer_hd
0,交通,0.42336562 -1.1400625 -2.0374057 -1.9194353 0....,-0.06610227 0.4988237 0.13218886 -0.77638644 1...,-0.012540849 -0.38462433 -0.32319498 -0.182240...,0.023853732 -0.09414571 -0.23024663 0.03881392...,-0.08571069687604904 0.05334413796663284 0.758...,-0.097999 -0.172532 0.674114 -0.103633 0.24910...,-0.0059281155 0.085107535 -0.018252835 0.06734...,2,0,...,10,3,-1,0,3,20,20,13,13,-1
1,房间,0.28802317 -0.4507514 0.35016647 -0.43055606 0...,0.12421269 0.024913808 -0.01602313 1.1140127 -...,0.19959436 -0.32069674 -0.14715314 0.061708618...,0.17186686 -0.013962293 -0.13654591 0.1969577 ...,-0.26072731614112854 0.6660227179527283 0.2643...,0.181897 0.389742 -0.06113 0.242304 -0.028627 ...,0.01048636 -0.015002572 -0.017642647 -0.040445...,24,17,...,20,11,32,0,27,6,15,5,20,31
2,面积,1.1007886 -0.69985676 0.23117578 -1.8015103 -1...,0.44881797 -0.6757507 0.74121505 0.08760744 -0...,0.2180409 -0.31570902 0.16386202 -0.18676051 -...,-0.0023564755 0.14263394 -0.27440792 0.0326909...,-0.4051162004470825 0.6650103330612183 0.01912...,0.311055 0.039625 0.333128 0.215967 -0.111913 ...,-0.06143476 -0.044589818 -0.019351065 0.036645...,16,16,...,1,29,19,0,7,31,0,28,-1,-1
3,布局,0.7827691 -0.91875976 -1.3275683 0.19824812 -0...,-0.8795247 -0.32223016 0.24907054 -0.6157134 -...,0.24405675 -0.41571265 -0.17313722 0.071001776...,0.15417029 -0.06498914 -0.15444419 -0.1932353 ...,0.0831306055188179 0.5076727867126465 0.217217...,0.266127 0.316716 0.021962 0.075516 0.086151 -...,-0.037475087 -0.01469476 -0.02024162 0.0472815...,16,30,...,0,32,-1,0,10,31,4,0,31,-1
4,卫浴,0.60243356 -0.9301406 -0.243199 -0.7276225 -0....,-0.12032523 0.20740871 0.35552785 0.07866025 -...,0.25761604 -0.07487154 -0.19477549 0.028649133...,0.19327797 0.098093584 0.0371605 -0.029816944 ...,-0.3666742742061615 0.4476194381713867 0.67427...,0.230336 -0.213324 0.503645 0.260184 0.126454 ...,0.028435966 -0.098250374 -0.019702565 -0.08827...,31,4,...,12,12,-1,0,9,13,29,19,12,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,三晚,-0.09600455 -0.24936976 0.08400188 0.089544974...,0.07487932 0.25357836 0.07852432 -0.0952891 0....,0.2406698 -0.5435254 0.10679261 0.0504834 -0.1...,0.123874895 -0.011844655 0.017515218 -0.050921...,0.22766144573688507 0.20426048338413239 0.3718...,-0.00788 0.306547 0.304058 0.261574 -0.089843 ...,0.005660566 -0.17151478 -0.012845519 0.0224691...,7,3,...,15,34,30,0,20,28,16,15,5,-1
936,礼貌,1.1424041 0.91151 0.43261242 0.6441254 0.60611...,-0.46472156 0.32605383 -0.73007 0.9956819 -0.6...,0.30669054 -0.12392349 0.01986696 0.033570386 ...,-0.029739557 -0.0032241284 -0.1473059 0.151833...,-0.09216073155403137 0.6346340179443359 0.4351...,-0.324412 0.066763 0.139822 0.105918 -0.272539...,0.035092603 0.1269345 -0.019277968 0.072855875...,18,28,...,17,19,2,0,19,21,11,0,10,-1
937,合适,-0.42161575 -0.19991678 1.7154243 -0.1863414 -...,0.836252 0.31240836 0.034565493 -0.46289173 0....,0.12132719 -0.42330444 0.15789798 0.16709346 0...,-0.21083379 0.13041505 -0.13174589 -0.09491256...,0.039119869470596313 0.3964913487434387 -0.013...,-0.266707 0.312565 -0.102081 0.449369 -0.25977...,0.021969374 0.031377662 -0.020527236 0.0181356...,21,12,...,23,7,3,0,25,2,10,3,34,35
938,威斯汀,-0.28125665 0.091284014 0.24053097 -0.07472294...,0.21199425 -0.13832602 -0.09146086 -0.09332652...,-0.3902142 -0.058192953 -0.1316207 -0.21724865...,0.15095249 0.24139097 -0.13767847 -0.099240825...,-0.4285953640937805 0.3718821406364441 0.07398...,-0.099088 0.133279 -0.222177 -0.11384 -0.31356...,-0.07128871 0.10444158 -0.018124782 0.08321853...,6,3,...,15,10,24,0,25,2,10,3,30,-1


In [18]:
metric

,metric,CBOW200_re_k,CBOW200_k,CBOW200_re_h,CBOW200_h,CBOW200_re_hd,CBOW200_hd,CBOW300_re_k,CBOW300_k,CBOW300_re_h,...,tencent200_re_h,tencent200_h,tencent200_re_hd,tencent200_hd,sentence_transformer_re_k,sentence_transformer_k,sentence_transformer_re_h,sentence_transformer_h,sentence_transformer_re_hd,sentence_transformer_hd
0,dvi,0.755357,1.868656,0.777130,1.944642,0.667484,1.008059,0.757305,1.899508,0.765027,...,0.866505,2.784710,0.741492,1.274969,0.755109,2.137338,0.815374,2.213769,0.708726,1.078133
1,sc,0.485533,0.062503,0.470005,0.054684,0.539126,0.247141,0.477971,0.054602,0.474909,...,0.386309,0.050869,0.451144,0.254959,0.467150,0.095414,0.436210,0.088782,0.473709,0.229595
2,chs,1690.527647,25.264119,1628.450094,24.344150,1521.843525,8.501300,1635.542042,24.824583,1626.440177,...,596.139335,12.858956,644.311414,13.452291,1055.220989,17.537720,970.092257,17.227861,978.633681,18.163454
3,class_number,35.000000,35.000000,35.000000,35.000000,32.000000,2.000000,35.000000,35.000000,35.000000,...,35.000000,35.000000,36.000000,2.000000,35.000000,35.000000,35.000000,35.000000,35.000000,37.000000


In [19]:
data.to_excel('../data/cluster_reduced_sentence.xlsx', index=False)
metric.to_excel('../data/metric_reduced_sentence.xlsx', index=False)